In [1]:
import os
import pandas as pd
import numpy as np

from tqdm.auto import tqdm
tqdm.pandas()


In [2]:
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY=os.environ["OPENAI_API_KEY"]

In [3]:
from langchain.embeddings import OpenAIEmbeddings

model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [4]:
df = pd.read_csv("ctgov_studies_20230621.csv", index_col="nct_id")

In [5]:
# Get all columns except overall_status
columns = list(df.columns)
columns.remove('overall_status')
columns

['brief_title',
 'official_title',
 'baseline_measurements',
 'brief_summaries',
 'detailed_descriptions',
 'criteria',
 'gender',
 'minimum_age',
 'maximum_age',
 'facilities',
 'city',
 'state',
 'zip',
 'country',
 'recruitment_details',
 'pre_assignment_details',
 'study_type']

In [6]:


df["concat"] = df[columns].apply(lambda x: ' '.join(
    x.astype(str) if x is not None else "None. "), axis=1)
df


,brief_title,official_title,overall_status,baseline_measurements,brief_summaries,detailed_descriptions,criteria,gender,minimum_age,maximum_age,facilities,city,state,zip,country,recruitment_details,pre_assignment_details,study_type,concat
nct_id,,,,,,,,,,,,,,,,,,,
NCT00000143,Studies of Ocular Complications of AIDS (SOCA)...,Studies of Ocular Complications of AIDS (SOCA)...,Completed,NaN,"To compare the newest CMV retinitis drug, cido...",Cytomegalovirus (CMV) is among the most freque...,Inclusion criteria:\n\nAge 13 years or older\n...,All,13 Years,NaN,"University of South Florida, MDC Box 21",Tampa,Texas,94143,United States,June 1997,NaN,Interventional,Studies of Ocular Complications of AIDS (SOCA)...
NCT00000378,Antidepressant Treatment of Melancholia in Lat...,Antidepressant Treatment of Melancholia in Lat...,Completed,NaN,The purpose of this study is to compare the sa...,To compare the efficacy and safety of a select...,Inclusion Criteria:\n\n-\n\nPatients must have...,All,60 Years,95 Years,1051 Riverside Drive,New York,New York,10032,United States,NaN,NaN,Interventional,Antidepressant Treatment of Melancholia in Lat...
NCT00000620,Action to Control Cardiovascular Risk in Diabe...,Action to Control Cardiovascular Risk in Diabe...,Completed,NaN,The purpose of this study is to prevent major ...,"BACKGROUND:\n\nCurrently, about 17 million Ame...",Inclusion Criteria:\n\nDiagnosed with type 2 d...,All,40 Years,79 Years,Wake Forest University,Winston-Salem,Washington,98195,United States,All participants had established type 2 diabet...,Eligible participants provided evidence of abi...,Interventional,Action to Control Cardiovascular Risk in Diabe...
NCT00001151,"Studies With 1,25-Dihydroxycholecalciferol","Studies With 1,25-Dihydroxycholecalciferol",Terminated,NaN,Vitamin D in the diet undergoes changes in the...,"Patients with extreme resistance to 1,25-dihyd...",INCLUSION CRITERIA:\n\nPatients with hereditar...,All,NaN,NaN,"National Institutes of Health Clinical Center,...",Bethesda,Maryland,20892,United States,NaN,NaN,Interventional,"Studies With 1,25-Dihydroxycholecalciferol Stu..."
NCT00001213,Cysteamine Eye Drops to Treat Corneal Crystals...,Trial of Topical Cysteamine in the Treatment o...,Completed,Although 328 participants were initially enrol...,Cystinosis is an inherited disease that result...,"Protocol 86-EI-0062 began as a randomized, dou...",INCLUSION CRITERIA:\n\nPatients must have a do...,All,2 Years,NaN,"National Institutes of Health Clinical Center,...",Bethesda,Maryland,20892,United States,NaN,NaN,Interventional,Cysteamine Eye Drops to Treat Corneal Crystals...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NCT05490771,Testing Copanlisib as a Potential Targeted Tre...,Phase II Study of Copanlisib in Patients With ...,"Active, not recruiting",NaN,This phase II MATCH treatment trial identifies...,PRIMARY OBJECTIVE:\n\nI. To evaluate the propo...,Inclusion Criteria:\n\nPatients must have met ...,All,18 Years,NaN,ECOG-ACRIN Cancer Research Group,Philadelphia,Pennsylvania,19103,United States,"Subprotocol Z1F was activated on June 20, 2018...",The PIK3CA mutations status was determined by ...,Interventional,Testing Copanlisib as a Potential Targeted Tre...
NCT05502081,Clinical Study to Compare Efficacy and Safety ...,Clinical Study to Evaluate the Possible Effica...,Completed,0. Uninfected\n\nAmbulatory mild disease\n\nAs...,Introduction:\n\nCorona Virus induced disease ...,I. INTRODUCTION\n\n1.1. COVID-19 overview and ...,Inclusion Criteria:\n\nage more than 12 years ...,All,12 Years,NaN,El-gomhoria St,Mansoura,El-dkhalia,050,Egypt,from 1/11/2021 to 29/5/2022 at isolation hospi...,assignment is applied after admission of parti...,Interventional,Clinical Study to Compare Efficacy and Safety ...
NCT05594173,Chewing and Oral Processing of Solid Food,Chewing and Oral Processing of Solid Food in H...,Completed,NaN,Food texture modification is commonly used as ...,Aim: To explore chewing a

In [7]:
import time

def _generate_embedding(
    text=None,
    model=None,
    amount=0,
):
    assert model is not None
    embedding = model.embed_documents([text])
    time.sleep(amount)
    embedding = np.array(embedding).squeeze()
    return embedding


In [8]:
df["embeddings"] = df["concat"].copy().astype(object)
# df["embeddings"] = df["concat"].progress_apply(
#     lambda row: _generate_embedding(text=row, model=model, amount=2)
# )

if not os.path.exists("npy"):
    os.makedirs("npy")

for index, row in tqdm(df.iterrows()):
    filename = f"npy/{index}.npy"
    embedding = _generate_embedding(text=row["concat"], model=model, amount=3)
    row["embeddings"] = embedding
    # Cache the embeddings for future use
    np.save(filename, embedding)


0it [00:00, ?it/s]

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 7becfc292d498dfc938364839b3e50ca in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 7becfc292d498dfc938364839b3e50ca in your message.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 7becfc292d49

In [9]:
df["embeddings"] = df["embeddings"].astype(object)
for index, row in tqdm(df.iterrows()):
    filename = f"npy/{index}.npy"
    embedding = np.load(filename)
    df["embeddings"].loc[index] = embedding.tolist()

0it [00:00, ?it/s]

In [10]:
df = df.drop(columns=["concat"], axis=1)
df.to_csv("ctgov_studies_20230621_openai_embedding.csv")


In [11]:
len(df["embeddings"][0])

1536

In [12]:
df["embeddings"][0][:5]


[-0.009765988955702291,
 0.0010155662695186536,
 0.02044236619934322,
 -0.020456159086402918,
 -0.0004319173955913906]

In [13]:
arr = np.load("npy/NCT00000143.npy")
arr 

array([-0.00976599,  0.00101557,  0.02044237, ..., -0.02474602,
       -0.01968371, -0.02139414])